In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
data = pd.read_csv("../input/train.csv")

In [ ]:
data['species'].describe()

In [ ]:
data.columns

In [ ]:
labels = data['species'].unique()
labels.sort()
labels

In [ ]:
for d in data.columns[2:]:
    data[d] = (data[d] - data[d].mean()) / data[d].std()

In [ ]:
data = data.drop('id', axis=1)
data

In [ ]:
temp = np.zeros(len(data['species']))
for i in range(len(data['species'])):
    temp[i] = np.where(labels == data['species'][i])[0]

temp

In [ ]:
data['species'] = temp
data['species']

In [ ]:
data['species'] = data['species'].astype(int)
data['species']

In [ ]:
X = data.drop((['species','margin8', 'margin16', 'margin34', 'margin61']), axis=1)
Y = data['species']
feature_names = X.columns

In [ ]:
feature_names

In [ ]:
from sklearn.cross_validation import train_test_split

In [ ]:
from sklearn import ensemble
rf = ensemble.RandomForestClassifier(n_estimators=1000, random_state=11)
rf.fit(X, Y)

In [ ]:
err_train = np.mean(Y != rf.predict(X))
print(err_train)

In [ ]:
a = X[:1]
print(rf.predict_proba(a))

In [ ]:
test = pd.read_csv("../input/test.csv")
id_col = test['id']
test = test.drop(['id','margin8', 'margin16', 'margin34', 'margin61'], axis=1)

In [ ]:
for t in test.columns:
    test[t] = (test[t] - test[t].mean()) / test[t].std()

In [ ]:
test.head()

In [ ]:
corr_df = test.corr() > 0.7
corr_df

In [ ]:
np_corr = corr_df.as_matrix()
np.fill_diagonal(np_corr,False)

In [ ]:
k = np.where(np_corr == True)
k = np.vstack(k)
k = np.stack((k[0],k[1]), axis = -1)
k[20:50]

In [ ]:
err_test = rf.predict_proba(test)

In [ ]:
temp = np.zeros(err_test.shape[0])
temp.shape

In [ ]:
result = pd.DataFrame(err_test, columns=labels)
result = pd.concat((id_col,result), axis=1)
result

In [ ]:
result.to_csv('result.csv', index=False)

In [ ]:
important = rf.feature_importances_
#important.sort()

In [ ]:
for i in zip(test.columns, important):
    if i[1] < 0.002 :
        print(i)